Todo:
- čísla týdnů (52+53 atd)
- správně jména! 
    - Žbirka Miro aj.

In [329]:
import os

In [330]:
import pandas as pd

In [331]:
folder = "data_raw"

In [332]:
df = pd.read_csv(os.path.join(folder,"ifpi_raw.csv"))

In [333]:
df

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel
0,0,CZ - ALBUMS - TOP 100,1. týden 2006,1,Album,PETR KOLÁŘ,UNIVERSAL MUSIC
1,1,CZ - ALBUMS - TOP 100,1. týden 2006,2,Music Bar,CHINASKI,UNIVERSAL MUSIC
2,2,CZ - ALBUMS - TOP 100,1. týden 2006,3,Čím to je?!,NO NAME,UNIVERSAL MUSIC
3,3,CZ - ALBUMS - TOP 100,1. týden 2006,4,44 slavných písniček,NEDVĚDI,UNIVERSAL MUSIC
4,4,CZ - ALBUMS - TOP 100,1. týden 2006,5,Best Of 1988-2005,PAVOL HABERA & TEAM,UNIVERSAL MUSIC
...,...,...,...,...,...,...,...
496801,45,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,46,Naše cesty,MAREK ZTRACENÝ,SUPRAPHON
496802,46,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,47,Naše hriechy (feat. Emma Drobná),ADAM ĎURICA,UNIVERSAL MUSIC
496803,47,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,48,Náš příběh (feat. Marek Lambora),TEREZA MAŠKOVÁ,WARNER MUSIC
496804,48,CZ - RADIO - TOP 50 CZ,Měsíc 9/2023,49,Patříme k sobě,OLGA LOUNOVÁ,OLGA LOUNOVÁ RECORDS


In [334]:
hitparady = df['hitparada'].drop_duplicates().to_list()
hitparady

['CZ - ALBUMS - TOP 100',
 'CZ - RADIO - TOP 100',
 'CZ - RADIO - TOP 20 MODERN ROCK',
 'CZ - ALBUMS CLASSICAL - TOP 10',
 'CZ - SINGLES DIGITAL - TOP 100',
 'CZ - SINGLES DIGITAL - TOP 50  CZ',
 'SK - ALBUMS - TOP 100',
 'SK - RADIO - TOP 100',
 'SK - RADIO - TOP 50 SK',
 'SK - SINGLES DIGITAL - TOP 100',
 'SK - SINGLES DIGITAL - TOP 50 SK',
 'CZ - RADIO - TOP 50 CZ']

## Opravy a čištění

### Aby byla data strojově čitelná

In [335]:
df['tyden'] = df['obdobi'].apply(lambda x: x.split(". t")[0] if ". t" in x else None)

In [336]:
df['tyden'] = df['tyden'].apply(lambda x: f"0{x}" if x and len(x) == 1 else x)

In [337]:
df['mesic'] = df['obdobi'].apply(lambda x: x.split("/")[0].split(" ")[1] if "Měsíc" in x else None)

In [338]:
df['mesic'] = df['mesic'].apply(lambda x: f"0{x}" if x and len(x) == 1 else x)

In [339]:
df['rok'] = df['obdobi'].apply(lambda x: x[-4:])

In [340]:
df.groupby('tyden')['track/album'].nunique().nsmallest(15)

tyden
5152       188
200903     190
200809     198
50.+51     206
53         328
52.+53     487
27.+28     519
30.+31     519
52.+01     524
52        1036
51        1785
51.+52    2072
50        4002
01        4108
31        4108
Name: track/album, dtype: int64

### Oprava chybných dat

In [341]:
df["rok"] = df["rok"].apply(lambda x: x.replace("223","2023").replace(" 201","2010").replace("2104","2014").strip())

## Pryč s dvojitými mezerami

In [342]:
df = df.apply(lambda x:x.replace("  "," "))

### Pryč s velkými písmeny (nebo jejich většinou)

In [343]:
slovodelice = [" ",".","/"]
for s in slovodelice:
    df['interpret'] = df['interpret'].apply(lambda x: s.join(word.title() for word in str(x).split(s)))
    df['vydavatel'] = df['vydavatel'].apply(lambda x: s.join(word.title() for word in str(x).split(s)))

In [344]:
df.sample(5)

,Unnamed: 0,hitparada,obdobi,pozice,track/album,interpret,vydavatel,tyden,mesic,rok
95812,62,CZ - RADIO - TOP 100,3. týden 2010,63,Evacuate The Dancefloor,Cascada,Universal Music,03,None,2010
448400,24,CZ - RADIO - TOP 50 CZ,9. týden 2007,25,Čas nejde vzít zpátky,David Koller,Universal Music,09,None,2007
121984,34,CZ - RADIO - TOP 100,12. týden 2015,35,Tuesday,I Love Makonnen,Warner Music,12,None,2015
414586,6,SK - SINGLES DIGITAL - TOP 100,16. týden 2022,7,Where Are You Now,Lost Frequencies & Calum Scott,Sony Music,16,None,2022
147133,83,CZ - RADIO - TOP 100,46. týden 2019,84,Holomráz,Slza,Universal Music,46,None,2019


### Sjednocení jmen

In [345]:
df[df['interpret'].str.lower().str.contains("hapka",na=False)]['interpret'].drop_duplicates().to_list()

['P.Hapka/M.Horáček',
 'Hapka/Horáček',
 'Hapka/Horáček/Brzobohatý',
 'Petr Hapka',
 'Petr Hapka A Michal Horáček',
 'Hapka Petr/Bílá Lucie',
 'Hapka Petr/Kirschner Jana',
 'Hegerová Hana/Hapka Petr']

In [346]:
spravna_jmena = {
    "M. Cyrus": "Miley Cyrus",
    "Ulrychovi Hana & Petr": "Hana A Petr Ulrychovi",
    "Ulrychovci":  "Hana A Petr Ulrychovi",
    "Miroslav Žbirka": "Miro Žbirka",
    "P.Hapka": "Petr Hapka",
    "Natalie Grossová": "Natálie Grossová",
    "Tichá Kateřina Marie": "Kateřina Marie Tichá",
    "Al Dhaba Gába": "Gabriela Al Dhábba",
    "Křížek Láďa": "Ladislav Křížek",
    " A Jiní": "",
    " A ": " a "
}

In [347]:
interpreti = df['interpret'].drop_duplicates().to_list()
[x for x in interpreti if len(x.split()) > 1 and f"{x.split()[1]} {x.split()[0]}" in interpreti]

['Nedvěd František',
 'Simon Paul',
 'Jones Norah',
 'Baumaxa Xavier',
 'Manu Chao',
 'Gahan  Dave',
 'Lipa Peter',
 'Duran Duran',
 'Sodoma Viktor',
 'Pospíšil Luboš & 5P',
 'Ward Shayne',
 'Toxxx Hugo',
 'Tobias Szidi',
 'Cohen Avishai',
 'Petřina Ota',
 'Janů Petra',
 'Pop Iggy',
 'Hrůza Michal & Ready Kirken',
 'Streisand Barbra',
 'Satriani Joe',
 'Bárta Dan & Robert Balzar Trio',
 'Knechtová Katarína & Peha',
 'Absolonová Monika',
 'Hybš Václav',
 'Spálený Jan',
 'Šporcl Pavel & Romano Stilo',
 'Ruml Ondřej',
 'Merta Vladimír & Etc',
 'Havelka Ondřej',
 'Dragoun Roman',
 'Smith  Patti',
 'Young Neil & Crazy Horse',
 'Brown Chris',
 'Mercury Freddie & Caballé Monserrat',
 'Horáček Michal & Lucia Šoralová',
 'Havelka Ondřej A Jeho Melody Makers',
 'Merta Vladimír & Dobrá Úroda',
 'Lang Lang',
 'Dědeček Jiří',
 'Bruni Carla',
 'Mišík Vladimír & Kovářík Mirek & Čdg',
 'Bonamassa Joe & Beth Hart',
 'Zagorová Hana / Petr Rezek',
 'Müller Richard & Fragile',
 'Petty Tom',
 'Tony Bennett

In [348]:
obracena_jmena = ["Vondráčková Lucie", "Jiřina Anna Jandová", "Smolík Jakub", "Sofian Medjmedj", "Josef Bohouš", "Müller Richard", "Zmožek Marcel", "Hrůza Michal", "Deyl David", "Bende Petr", "Zonyga Jiří", "Hapka Petr", "Nedvěd František", "Mrózek Daniel", "Ondra Jakub", "Smejkal Jaro", "Vondráčková Helena", "Střihavka Kamil", "Charvátová Jitka", "Goldová Gabriela", "Jan J. Nedvěd Vs. Bek Ofis", "Daniel Šafařík", "Jandová Marta", "Horváth Vlastimil", "Horecká Hana", "Farna Ewa", "John Wolfhooker", "Adam Janota",  "Dyk Vojtěch", "Jíšová Pavlína", "Sanchez Yvonne", "Ruppert Matěj", "Burian Michael", "Brzobohatá Katka", "Václav Rouček", "Šinkorová Daniela", "Šeps Michal", "Absolonová Monika", "Zíma Josef", "Ztracený Marek", "Zemanová Bára", "Bastien Alan", "Maxa Martin", "Matuš Bohuš", "Hron Vláďa", "Hudček Michal", "Háma Aleš", "Hübsch Kamila", "Hůlka Daniel", "Milfajtová Sára", "Krajčo Richard", "Kraus David", "Kreyson Memorial", "Krézlová Karolína", "Josef Vágner, Charlotte Gottová", "Hladík Radim", "Peter Fider", "Drda Zbyněk", "Paul Project", "Povýšilová Monika", "Poláček Petr", "Poliak Michal", "Hanák Tomáš", "Hayek Martin", "Nekonečný Dan", "Neckář Václav", "Novotný Miloš","Rosová Sandra","Vojtek Roman","Bagová Ivanna","Kerndlová Tereza","Kotvald Petr",'Simon Paul',
 'Jones Norah',
 'Baumaxa Xavier',
 'Chao Manu',
 'Gahan Dave',
 'Lipa Peter',
 'Pospíšil Luboš',
 'Sodoma Viktor',
 'Ward Shayne',
 'Toxxx Hugo',
 'Tobias Szidi',
 'Cohen Avishai',
 'Petřina Ota',
 'Janů Petra',
 'Pop Iggy',
 'Streisand Barbra',
 'Satriani Joe',
 'Bárta Dan',
 'Knechtová Katarína',
 'Hybš Václav',
 'Spálený Jan',
 'Šporcl Pavel',
 'Ruml Ondřej',
 'Merta Vladimír',
 'Havelka Ondřej',
 'Dragoun Roman',
 'Smith Patti',
 'Young Neil',
 'Brown Chris',
 'Mercury Freddie',
 'Caballé Monserrat',
 'Horáček Michal',
 'Havelka Ondřej',
 'Merta Vladimír',
 'Dědeček Jiří',
 'Bruni Carla',
 'Mišík Vladimír',
 'Bonamassa Joe',
 'Zagorová Hana',
 'Petty Tom',
 'Bennett Tony',
 'Dusilová Lenka',
 'Ondřej Havelka',
 'Ruml Ondřej',
 'Szidi Tobias',
 'Ondřej Havelka',
 'Roman Dragoun And His Angels',
 'Avishai Cohen Trio',
 'Sinatra Frank',
 'Zdeněk Bína',
 'Melody Gardot',
 'Petřina Ota',
 'Cooper Alice',
 'Satriani Joe',
 'Veverka Vilém',
 'Bernstein Leonard',
 'Dragoun Roman',
 'Clyro Biffy',
 'Gardot Melody',
 'Spálený Jan',
 'Hybš Václav',
 'Michal Hromek',
 'Bernstein Leonard',
 'Veverka Vilém',
 'Svěcený Jaroslav',
 'Semerádová Jana',
 'Ramazzotti Eros & Anastacia',
 'Foret Michael',
 'Ward Shayne',
 'Keys Alicia & White Jack',
 'Burke Alexandra Feat. Flo Rida',
 'Bieber Justin',
 'Tempah Tinie',
 'Williams Robbie',
 'Iglesias Enrique',
 'Solveig Martin',
 'Lopez Jennifer',
 'Sinclar Bob',
 'Posner Mike',
 'Brown Chris',
 'Solveig Martin',
 'Lopez Jennifer',
 'Brown Chris',
 'Gomez Selena',
 'Murs Olly',
 'Harris Calvin',
 'Burke Alexandra',
 'Konvičková Markéta',
 'Harris Calvin',
 'Grey Skylar',
 'Romero Nicky',
 'Solveig Martin',
 'Derulo Jason',
 'Tempah Tinie',
 'Lavigne Avril',
 'Harris Calvin',
 'Bublé Michael',
 'Aristone Peter',
 'Garand Paulie',
 'Azalea Iggy',
 'Paul Sean',
 'Ponte Gabry',
 'Cooper Alice',
 'Bína Zdeněk',
 'Bezdeda Tomáš',
 'Opatovský Robo',
 'Kerndlová Tereza',
 'Kmeťo Igor',
 'Lipa Peter',
 'Tomáš Bezdeda',
 'Mirgová Dominika',
                 'Gahan Dave',
                 'Cohen Avishai',
                 'Smith Patti',
                 'Dragoun Roman',
                 'Gott Karel']

In [349]:
for o in obracena_jmena:
    spravna_jmena[o] = f"{o.split()[1]} {o.split()[0]}"

In [350]:
for key, val in spravna_jmena.items():
    df['interpret'] = df['interpret'].apply(lambda x: x.replace(key,val))

In [351]:
interpreti = df['interpret'].drop_duplicates().to_list()
[x for x in interpreti if len(x.split()) > 1 and f"{x.split()[1]} {x.split()[0]}" in interpreti]

['Gahan  Dave',
 'Duran Duran',
 'Avishai Cohen',
 'Roman Dragoun',
 'Smith  Patti',
 'Lang Lang',
 'Dragoun Roman',
 'Cohen Avishai',
 'Michal Prokop & Framus 5',
 'Jiří Suchý',
 'Michal Prokop a Framus 5',
 'Jiří Suchý & Jitka Molavcová',
 'Laurie Anderson & Kronos Quartet',
 'Jan Vodňanský a Haňa Navarová',
 'Langerová Aneta / Vojtek Radim',
 'Guetta David Vs. The Egg',
 'Guetta David Feat. Rowland Kelly ',
 'Guetta David Feat. Akon',
 'Guetta David Feat. Fergie',
 'Guetta David Feat. Rihanna',
 'Bílá Lucie Feat. Vojtek Josef',
 'Guetta David Feat. Flo-Rida & Nicki Minaj',
 'Guetta David Feat.Taio Cruz & Ludacris',
 'Guetta David Feat. Sia',
 'Guetta David Feat. Usher',
 'David Michal a Lucie Vondráčková',
 'Guetta David & Minaj Nicki',
 'Suchý Jiří',
 'Guetta David Feat. Chris Brown And Lil Wayne',
 'Timberlake Justin & Jay-Z',
 'Guetta David Feat. Taped Rai',
 'Guetta David & Ne-Yo & Akon',
 'Guetta David Feat. Skylar Grey',
 'Junge Junge & Kyle Pearce',
 'Junge Junge',
 'Junge Ju

### Hlavní intepreti

In [352]:
df[df['interpret'].str.contains(" a ")]['interpret'].drop_duplicates()

86                  Jaroslav Uhlíř a Zdeněk Svěrák
390                          Hana a Petr Ulrychovi
874                                  Mňága a Žďorp
9590                          Ryvolovi Wabi a Miky
12990                               Žalman a Spol.
14050                 Robert Křesťan a Druhá Tráva
17784          Havelka Ondřej a Jeho Melody Makers
21084                      Michnová Zuzana a Hosté
23460                   Wabi Daněk a Ďáblovo Stádo
24147                  Petr Hapka a Michal Horáček
28567                  Věra Špinarová a Petra Janů
30456                           Laura a Její Tygři
34097                     Michal Prokop a Framus 5
58233                       Y.Simonová a M.Chladil
60492                Jan Vodňanský a Haňa Navarová
76104                             System Of a Down
88042                                Marek a Kamil
104510            David Michal a Lucie Vondráčková
108147                   Markéta Konvičková a Elis
110710    Eddie Stoilow Feat. E

In [353]:
def kdojeprvni(x):
    delice = ["&","/","(feat.","feat.","Feat."," a "]
    for d in delice:
        if d in x:
            x = x.split(d)[0]
    return x.strip()

In [354]:
df['interpret_hlavni'] = df['interpret'].apply(lambda x: kdojeprvni(x))

In [355]:
df['interpret_hlavni'].drop_duplicates().to_list()

['Petr Kolář',
 'Chinaski',
 'No Name',
 'Nedvědi',
 'Pavol Habera',
 'Aneta Langerová',
 'Těžkej Pokondr',
 'Robbie Williams',
 'Damichi',
 'Divokej Bill',
 'Petr Novák',
 'Různí',
 'Vlasta Horváth',
 'Rammstein',
 'Madonna',
 'Petr Bende',
 'Helena Zeťová',
 'Jakub Smolík',
 'Čechomor',
 'Richard Müller',
 'Tatu',
 'Karel Gott',
 'Depeche Mode',
 'Petr Muk',
 'Lenka Filipová',
 'Michal David',
 'Black Eyed Peas',
 'Anastacia',
 'Juanes',
 'Amiri Ali',
 'Issa Sámer',
 'Tři Sestry',
 'Věra Špinarová',
 'Enya',
 'Karel Kryl',
 'Maxim Turbulenc',
 'Miro Žbirka',
 'Zuzana Navarová',
 'Iveta Bartošová',
 'Marie Rottrová',
 'Pražský Výběr',
 'Hana Hegerová',
 'Dan Bárta',
 'Iva Frühlingová',
 'Jiří Schelinger',
 'Leoš Mareš',
 'Janek Ledecký',
 'Eminem',
 'Jaroslav Uhlíř',
 'Petr Hapka',
 'Lucie Vondráčková',
 'Alanis Morissette',
 '4Tet',
 'Jana Kirschner',
 'James Blunt',
 'Crazy Frog',
 'Lenka Dusilová',
 'Victoria',
 'Hana Zagorová',
 'Marika Gombitová',
 'Radůza',
 '2M',
 'Peha',
 'Pro

In [356]:
len(df[(df['hitparada'] == 'CZ - RADIO - TOP 100')]['interpret_hlavni'].drop_duplicates().to_list())

1439

### Album/track

In [357]:
df['album'] = df.apply(lambda row: row['track/album'] if 'ALBUM' in row['hitparada'] else None, axis = 1)

In [358]:
df['track'] = df.apply(lambda row: row['track/album'] if 'ALBUM' not in row['hitparada'] else None, axis = 1)

In [359]:
df = df.drop(columns=['track/album','obdobi'])

In [360]:
poradi = ['hitparada','rok','mesic','tyden','pozice','album','track','interpret','interpret_hlavni','vydavatel']

In [361]:
df = df.reindex(columns=poradi)

### Seřazení

In [362]:
df = df.sort_values(by=['hitparada','rok','mesic','tyden','pozice'])

In [363]:
df.sample(10)

,hitparada,rok,mesic,tyden,pozice,album,track,interpret,interpret_hlavni,vydavatel
367208,SK - RADIO - TOP 50 SK,2018,None,40,15,None,Chýbanie,Desmod,Desmod,Orpheus Production
146797,CZ - RADIO - TOP 100,2019,None,44,48,None,Birds,Imagine Dragons,Imagine Dragons,Universal Music
341243,SK - RADIO - TOP 100,2020,None,30,75,None,Born To Be Yours,Kygo,Kygo,Sony Music
436215,SK - SINGLES DIGITAL - TOP 50 SK,2020,None,41,40,None,Neviditelnej (feat. Viktor Sheen & Calin),Kontrafakt,Kontrafakt,Tvoj Tatko Records
440261,SK - SINGLES DIGITAL - TOP 50 SK,2022,None,07,36,None,Nie,Dms,Dms,Ruka Hore
226184,CZ - SINGLES DIGITAL - TOP 100,2019,05,None,66,None,Here With Me (feat. CHVRCHES),Marshmello,Marshmello,Universal Music
471008,CZ - RADIO - TOP 50 CZ,2015,None,30,36,None,Žijeme len raz,Ego Feat. Robert Burian,Ego,Warner Music
12620,CZ - ALBUMS - TOP 100,2010,None,48,21,Už z hor zní zvon,None,Různí,Různí,Supraphon
4237,CZ - ALBUMS - TOP 100,2007,None,35,38,Romano hip hop,None,Gipsy.Cz,Gipsy.Cz,Ir
104538,CZ - RADIO - TOP 100,2011,None,40,89,None,We Found Love,Rihanna & Calvin Harris,Rihanna,Universal Music


### Export do jednotlivých dataframů

In [364]:
import os

if not os.path.exists("data"):
    os.mkdir("data")
else:
    pass

In [365]:
for h in hitparady:
    df_p = df[df["hitparada"] == h].reset_index(drop=True)
    if "ALBUMS" in h:
        df_p = df_p.drop(columns=['track'])
    else:
        df_p = df_p.drop(columns=['album'])
    df_p2 = df_p[~df_p['mesic'].isnull()]
    h = h.replace(" ","_").replace("-_","")
    if len(df_p2) > 0:
        df_p2 = df_p2.drop(columns=['tyden'])
        df_p2.to_csv(os.path.join("data",f"{h}_mesicni.csv"))
    df_p2 = df_p[~df_p['tyden'].isnull()]
    if len(df_p2) > 0:
        df_p2 = df_p2.drop(columns=['mesic'])
        df_p2.to_csv(os.path.join("data",f"{h}_tydeni.csv"))